In [1]:
import sys
from pathlib import Path

# 프로젝트 루트를 Python 경로에 추가 (rag 모듈 import를 위해 필요)
project_root = Path().resolve().parent  # 현재가 LangGraph 폴더이므로, parent가 프로젝트 루트
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ 프로젝트 루트 추가됨: {project_root}")

✅ 프로젝트 루트 추가됨: C:\Users\rkddn\YAI-NLP


컬렉션 정보 + 문서 목록

In [4]:
import chromadb

client = chromadb.PersistentClient(path="./chroma_db")

print("📂 컬렉션 목록:", [c.name for c in client.list_collections()])

collection = client.get_collection("web_search")
print(f"📚 문서 수: {collection.count()}")

📂 컬렉션 목록: ['web_search', 'my_collection']
📚 문서 수: 11


저장된 문서 내용 보기

In [7]:
result = collection.peek(limit=10)

results = collection.get(
    limit=10,
    include=["metadatas", "documents", "embeddings"],
    where={"source": "highmath12.txt"}
)

for i, (doc, meta) in enumerate(zip(results["documents"], results["metadatas"])):
    print(f"\n--- 청크 {i+1} ---")
    print(f"메타데이터: {meta}")
    print(f"내용: {doc[:200]}---")

특정 키워드로 검색 테스트

In [8]:
from rag.base import create_embedding_auto

embedding = create_embedding_auto()
query_vector = embedding.embed_query("팔란티어에 대해 설명해줘")

results = collection.query(
    query_embeddings=[query_vector],
    n_results=3,
    include=["metadatas", "documents", "distances"]
)

for i, (doc, meta, dist) in enumerate(zip(
    results["documents"][0], results["metadatas"][0], results["distances"][0]
)):
    print(f"\n--- 결과 {i+1} (거리: {dist:.4f}) ---")
    print(f"출처: {meta}")
    print(f"내용: {doc[:300]}")

🌐 임베딩: API 방식 (BAAI/bge-m3)

--- 결과 1 (거리: 0.7221) ---
출처: {'source': 'web_search:팔란티어에 대해 설명해줘', 'origin': 'travily_merged'}
내용: 팔란티어(Palantir)의 빅데이터 분석 기술, 고담 플랫폼 활용 사례, 최근 주가 상승과 미래 전망을 다룬 종합적인 분석을 다룹니다. 이 이름은 ‘넓은 범위를 감시하거나 미래를 볼 수 있는 천리안의 구슬’을 뜻하며, 빅데이터 분석과 보안 소프트웨어에 특화된 팔란티어의 정체성을 잘 보여줍니다. 팔란티어는 대규모 데이터를 효율적으로 관리하고 분석하는 플랫폼을 통해, 미국 중앙정보국(CIA), 연방수사국(FBI)과 같은 주요 정부 기관 및 기업 고객들에게 솔루션을 제공하고 있습니다. ## 팔란티어의 핵심 기술. 팔란티어의 성공은

--- 결과 2 (거리: 0.7484) ---
출처: {'origin': 'travily_merged', 'source': 'web_search:팔란티어에 대해 설명해줘'}
내용: 팔란티어는 종종 데이터 브로커, 데이터 마이너, 혹은 거대한 개인정보 데이터베이스라고 불린다. 실상 팔란티어는 이 중 어느 것에도 해당하지
출처: https://ebadak.news/2025/08/13/palantir-what-the-company-does/

---

팔란티어(Palantir Technologies)는 2003년에 설립된 데이터 분석 및 통합 플랫폼 기업으로, 주로 정부와 민간 부문에서 데이터 기반 의사결정을 지원하는 고급 소프트웨어
출처: https://brunch.co.kr/@minjumin/17

---

--- 결과 3 (거리: 0.8333) ---
출처: {'origin': 'travily_merged', 'source': 'web_search:팔란티어에 대해 설명해줘'}
내용: 오늘도 지난 컨텐츠와 마찬가지로 AI 소프트웨어 산업과 기업들 이야기좀 해보려고 하는데요, 요즘 미국 증시에서 가장 핫한 기업

메타데이터 기준 필터링

In [16]:
# 특정 소스 파일의 문서만 보기
results = collection.get(
    where={"source": "nlp.pdf"},     # 메타데이터 필터
    include=["documents", "metadatas"]
)
print(f"nlp.pdf 청크 수: {len(results['documents'])}")

# 특정 페이지만 보기
results = collection.get(
    where={"page": 1},
    include=["documents", "metadatas"]
)

nlp.pdf 청크 수: 150


Summarization

In [18]:
# 컬렉션 전체 통계
all_data = collection.get(include=["metadatas"])
sources = set(m.get("source", "unknown") for m in all_data["metadatas"])

print(f"📊 컬렉션 요약")
print(f"  총 청크 수: {collection.count()}")
print(f"  소스 파일: {sources}")
for src in sources:
    count = sum(1 for m in all_data["metadatas"] if m.get("source") == src)
    print(f"    - {src}: {count}개 청크")

📊 컬렉션 요약
  총 청크 수: 182
  소스 파일: {'highmath12.txt', 'nlp.pdf'}
    - highmath12.txt: 32개 청크
    - nlp.pdf: 150개 청크
